## Introduction to Pytorch Code Examples

이 노트는 Pytorch 딥러닝 프로젝트 구조화에 대한 training, model, loss function, optimzer를 전반적으로 다루는 내용입니다. [CS230 link](https://cs230.stanford.edu/blog/pytorch/)

CS230 Project Code Example로 발표한 [메인](https://cs230.stanford.edu/blog/tips/) 포스트에 대한 후속 내용이며 CS230 Project Code Example의 소스 코드는 [github 레파지토리](https://github.com/cs230-stanford/cs230-code-examples)에서 확인 가능합니다.
```
pytorch/
    vision/
    nlp/
```

이 튜토리얼은 아래 코드 예제 시리즈 중 하나입니다
* [설치, 프로젝트 시작](https://cs230.stanford.edu/blog/tips/)
* (이 노트에 해당) Pytorch code의 전반적인 구조
* [hand sign 이미지에 대한 레이블 예측](https://cs230.stanford.edu/blog/handsigns/)
* [개체명인식(NER)](https://cs230.stanford.edu/blog/namedentity)

## 이 튜토리얼의 목표
* Pytorch에 대한 이해
* 어떻게 정확하게 Pytorch 딥러닝 프로젝트 구조화하는지에 대한 예제
* 필요에 맞는 수정을 하기 위하여 잘 짜여진 코드 이해하기

## 참고 자료
* [Pytorch 사이트](https://pytorch.org/)
* [Pytorch 공식 튜토리얼](https://pytorch.org/tutorials/)은 다양한 유스케이스들을 다룹니다.(ex. attention based seq2seq model, Deep Q-Net, Neural transfer 등)
* [Pytorch 60분 리뷰](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)
* Justin Johnson의 [git repository](https://github.com/jcjohnson/pytorch-examples), 자체 예제를 통해 Pytorch 컨셉들에 대한 기초적 부분을 소개합니다.
* [그 밖에](https://github.com/ritchieng/the-incredible-pytorch)

## 코드 레이아웃
모든 Pytorch 예제 코드는 아래 공통적인 구조를 공유합니다

```
data/
experiments/
model/
    net.py
    data_loader.py
train.py
evaluate.py
search_hyperparams.py
synthesize_results.py
evalutate.py
utils.py
```
* `model/net.py` : 뉴럴 네트워크 구조와 손실 함수 그리고 평가 지표을 지정합니다
* `model/data_loader.py`: 데이터가 네트워크에 어떻게 피드되는지
* `train.py`: 메인 학습 loop 내용을 포함합니다
* `evaluate.py`: 모델 평가를 위한 메인 loop 내용을 포함합니다
* `utils.py`: 하이퍼파라미터, 로깅, 모델 저장에 대한 유용한 함수들

이해를 위해 `train.py`를 먼저 읽기를 추천합니다

앞으로 업무 태스크와 데이터셋을 통해 프로젝트가 어느정도 이해가 되면, 아마 아래 내용을 수정을 하고 싶을 수도 있습니다
* `model/net.py`: 모델 변경, 즉 입력을 예측과 손실로 변환하는 방법 등
* `model/data_loader.py`: 모델에 데이터를 공급하는 방식을 변경
* `train.py`와 `evaluate.py`: 필요하다면, 각각의 문제에 맞는 구체적인 변경

## 텐서와 변수

시작전에, pytorch 기초에 대한 이해를 얻기 위하여 [Pytorch 60분 리뷰](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)를 먼저 보시는걸 강력히 추천합니다. 아래는 맛보기 샘플입니다.

Pytorch 텐서는 Numpy array와 동작이 비슷합니다

In [2]:
import torch

a = torch.Tensor([[1,2],[3,4]])
print(a)

tensor([[1., 2.],
        [3., 4.]])


In [3]:
print(a**2)

tensor([[ 1.,  4.],
        [ 9., 16.]])


Pytorch 변수는 텐서를 감싸고 텐서에 대하여 실행할 동작을 설정할 수 있습니다. 아래는 자동 미분을 실행하도록 한 것입니다.

In [5]:
from torch.autograd import Variable

a = Variable(torch.Tensor([[1,2],[3,4]]), requires_grad=True)
print(a)

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)


In [7]:
y = torch.sum(a**2)
print(y)

tensor(30., grad_fn=<SumBackward0>)


In [8]:
y.backward() 

In [9]:
print(a.grad) 

tensor([[2., 4.],
        [6., 8.]])


PyTorch는 미니멀하고 직관적인 구문으로 우아함과 표현력을 제공합니다. 계속 진행하기 전에 "참고 자료" 섹션 의 몇 가지 예를 더 보기를 추천합니다

## 핵심 학습 단계
학습 알고리즘의 핵심이 어떻게 보이는지 살펴 봅니다. 아래의 다섯 줄은 모델에 batch 입력을 전달하고 손실을 계산하며 역전파를 수행하고 파라미터를 업데이트합니다.

In [ ]:
output_batch = model(train_batch)           # 모델 출력 셰산
loss = loss_fn(output_batch, labels_batch)  # 손실 계산

optimizer.zero_grad()  # 이전 그래디언트들 초기화
loss.backward()        # 모든 변수 loss에 대한 그래디언트 계산

optimizer.step()       # 계산된 그래디언트를 사용하여 업데이트 수행

각 변수 `train_batch`, `labels_batch`, `output_batch` 및 `loss` 는 PyTorch 변수이고 미분이 자동적으로 계산되도록 합니다.

우리가 작성한 다른 모든 코드는 모델의 정확한 사양, 데이터와 레이블의 batch를 가져오는 방법, 손실 계산 및 옵티마이저의 상세내용을 중심으로 작성되었습니다. 이 포스트에서 PyTorch에서 간단한 모델을 작성하고 손실을 계산하고 최적화 프로그램을 정의하는 방법을 설명합니다. 다음 포스트는 각각 이미지 데이터와 텍스트 데이터에 대한 데이터 가져오는 부분을 다룹니다.

## Pytorch 모델

모델은 Pytorch에서 `torch.nn.Module 클래스`를 상속받음으로써 정의될 수 있습니다. 이 모델은 2단계로 정의됩니다. 우리는 먼저 모델의 파라미터들을 정의하고 파라미터들이 어떻게 인풋에게 적용되는지 정의합니다. 학습하는 파라미터가 포함되어 있지 않은 작업들을 위해(예를들어 ReLU와 같이 activation fuction이나 maxpool같은 동작들), `torch.nn.functional` 모듈을 일반적으로 사용합니다. 여기 single hidden layer 네트워크를 통한 예제가 있습니다.

In [1]:
import torch.nn as nn
import torch.nn.functional as F

class TwoLayerNet(nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        여기 생성자에서 멤벼변수로 두개의 nn.Linear 모듈을 생성하고 할당합니다

        D_in: 입력 차수
        H: 히든 레이어의 차수
        D_out: 출력 차수
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = nn.Linear(D_in, H) 
        self.linear2 = nn.Linear(H, D_out)

def forward(self, x):
        """
        여기서 input data 변수를 입력으로 받고, output 변수를 리턴합니다.
        변수에 대한 추상화 오퍼레이터 뿐만아니라 생성자에서 정의된 모듈들을 여기서 사용 가능합니다
        """
        h_relu = F.relu(self.linear1(x))
        y_pred = self.linear2(h_relu)
        return y_pred

이 `__init__` 함수는 모델의 두 리니어 레이어를 초기화합니다. PyTorch는 우리가 지정한 파라미터에 대하여 적절한 초기화 처리를 합니다.

`forward` 함수에서는 먼저 첫 번째 선형 레이어를 적용하고, ReLU 활성화를 적용한 다음 두 번째 선형 레이어를 적용합니다.

`x`의 첫 번째 차원이 배치 크기 라고 가정합니다. 네트워크에 대한 입력이 단순한 차원 100의 벡터이고 배치 크기가 32이면 `x`에 대한 차원은 32,100이 되겠습니다. 모델을 정의하고 forward로 계산하는 방법의 예제를 살펴 보겠습니다.

In [ ]:
# N은 배치 사이즈
# D_in 은 인풋 차수
# H는 히든 레이어의 차수
# D_out은 출력 차수
N, D_in, H, D_out = 32, 100, 50, 10

# input과 output을 가질 랜덤 텐서를 생성하고 변수로 래핑합니다
x = Variable(torch.randn(N, D_in))  # 차수: 32 x 100

# 위에서 정의한 클래스를 인스턴스하여 모델을 생성합니다
model = TwoLayerNet(D_in, H, D_out)

# Forward 계산: x를 모델에 입력하여 예측값 y를 계산합니다
y_pred = model(x)   # 차수: 32 x 10

더 복잡한 모델들도 같은 레이아웃을 따르며 그들중 두개를 이후 포스트에서 다룰겁니다

## LOSS FUNCTION

PyTorch는 `torch.nn` [모듈](https://pytorch.org/docs/master/nn.html#loss-functions)에서 많은 표준 손실 함수들을 제공합니다. 교차 엔트로피 손실을 계산하는 방법에 대한 간단한 예는 다음과 같습니다. 모델이 `C` 레이블의 다중 클래스 분류 문제를 해결한다고 가정해 보겠습니다. batch 사이즈인 `N`에 대하여 모델에 인풋 배치를 입력으로 넘기면서 얻어지는 `out`은 `NxC`차수의 Pytorch 변수입니다. 또한 `N` 사이즈인 각각 요소가 해당 예제의 정답인(ex. `[0,...,C-1]`의 레이블) `target` 변수가 있습니다. 손실 함수를 정의하고 다음과 같이 손실을 계산할 수 있습니다.

loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(out, target)

Pytorch는 이와 같은것들을 확장하고 커스텀한 손실 함수를 작성하는걸 매우 쉽게 해줍니다. 우리는 아래와 같이 커스텀한 Cross Entropy Loss 함수를 작성합니다. (Numpy식 코드에 주의하세요)

In [17]:
def myCrossEntropyLoss(outputs, labels):
    batch_size = outputs.size()[0]            # batch_size
    outputs = F.log_softmax(outputs, dim=1)   # softmax log 계삱
    outputs = outputs[range(batch_size), labels] # label에 해당하는 값 가져오기
    return -torch.sum(outputs)/num_examples

이건 꽤 심플한 커스텀한 loss 함수 작성 예제였습니다. [NLP](https://cs230.stanford.edu/blog/namedentity/) 섹션에서 우리는 흥미로운 커스텀 loss 함수를 살펴볼것입니다

## OPTIMIZER

`torch.optim` 패키지는 공통 최적화 알고리즘들에 대한 사용하기 쉬운 인터페이스를 제공합니다. 옵티마이저를 정의하는것은 아래와 같이 매우 쉽습니다.

In [18]:
#pick an SGD optimizer
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)

#or pick ADAM
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

매 iterate 마다 업데이트가 필요한 모델의 파라미터를 전달합니다. 계층 별 또는 파라미터별 learning rate 같은보다 복잡한 방법을 지정할 수도 있습니다.

`loss.backward()`를 사용하여 그래디언트들이 계산하면, `optimizer.step()`은 이 파라미터들을 정의된 최적화 알고리즘 대로 업데이트합니다

## 학습 vs 평가

모델 학습 전에, `model.train()` 호출은 반드시 해야되는 것입니다. 이와 마찬가지로, `model.eval()`도 모델 테스트 전에 반드시 호출해야합니다. 이를 통해 교육 및 테스트 동안 dropout과 배치 정규화가 수정됩니다.

## 메트릭스 계산

이 단계까지 `train.py`와 `evaluate.py`의 대부분의 코드를 이해할 수 있어야합니다. (데이터를 어떻게 가져올지는 제외하고, 이는 이 후 포스트에서 다룰 겁니다). 손실을 모니터링 한다는 것 외에, 정확도나 precision/recall 과 같은 다른 메트릭들을 모니터링 하는 것도 도움이 됩니다. 이를 위해, `model/net.py`에서 모델 출력 배치를 위한 커스텀한 메트릭 함수들을 정의합니다. 더 쉽게하기 위해, 메트릭 함수에 전달 전에 Pytorch 변수를 Numpy array로 변환합니다. Loss Function 섹션에서 설정한것 처럼 멀티 분류 문제를 위해 Numpy를 사용하여 아래와 같이 정확도를 계산하는 함수를 작성할 수 있습니다.

In [19]:
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs==labels)/float(labels.size)

`model/net.py`에 커스텀한 메트릭들을 추가할 수 있습니다. 추가하면, 단순하게 `metrics` 딕셔너리에 이들을 추가할 수 있습니다.

In [21]:
metrics = { 'accuracy': accuracy,
            ## 커스텀 메트릭들을 추가,
          }

## 모델 저장 및 불러오기

모델을 저장하고 불러오는 유틸 함수들을 `utils.py`에 정의합니다. 모델을 저장하기 위해 아래와 같이 호출해야 됩니다.

In [ ]:
state = {'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'optim_dict' : optimizer.state_dict()}
utils.save_checkpoint(state,
                      is_best=is_best,      # 만약 이 모델이 베스트 모델이라면 True
                      checkpoint=model_dir) # path to folder

`utils.py`는 정의한 state 함수를 저장하기 위해 `torch.save(state, filepath)` 함수를 내부적으로 사용합니다. 이 사전안에 다른 것을 추가할 수 있습니다. `model.state_dict()`는 모델의 파라미터들을 저장합니다. 그리고 `optimizer.state_dict()`는 옵티마이저의 상태를(예를들어 러닝레이트 값과 같은) 저장합니다.

어떤 체크포인트로부터 저장된 상태들을 불러오기위해 아래와 같이 호출하면 됩니다.

In [ ]:
utils.load_checkpoint(restore_path, model, optimizer)

`optimizer` 매개변수는 선택입니다. 그리고 새로운 optimizer를 사용해 다시시작 할 수도 있습니다.
`load_checkpoint`는 저장했던 체크포인트를 내부적으로 불러옵니다. 그리고 모델 weight들과 이 optimizer의 state를 복원합니다. 

## GPU 사용하기

In [ ]:
model = net.Net(params).cuda() if params.cuda else net.Net(params)

In [ ]:
if params.cuda:
    batch_data, batch_labels = batch_data.cuda(), batch_labels.cuda()

Pytorch는 위의 명령문들을 가지고 GPU 사용이 명쾌하고 명료하게 합니다.  모델이나 텐서, 변수 위에 `.cuda()`를 호출은 모델/텐서/변수를 GPU로 보냅니다. GPU에서 모델을 학습하기 위해, 관련있는 모든 파라미터와 변수들은 반드시 `.cuda()`를 호출해 GPU로 보내져야합니다.

## 결론

더불어 [Vision](https://cs230.stanford.edu/blog/handsigns/) 예제나 [NLP](https://cs230.stanford.edu/blog/namedentity/) 에제를 진행할 수 있습니다.